In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [71]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data preprocessing and augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.bn1(self.conv1(x))))
        x = self.pool(nn.functional.relu(self.bn2(self.conv2(x))))
        x = self.pool(nn.functional.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

Files already downloaded and verified
Files already downloaded and verified


In [72]:
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.3f}")

print("Training finished.")

Epoch 1, Loss: 1.569
Epoch 2, Loss: 1.252
Epoch 3, Loss: 1.105
Epoch 4, Loss: 1.019
Epoch 5, Loss: 0.939
Epoch 6, Loss: 0.883
Epoch 7, Loss: 0.848
Epoch 8, Loss: 0.807
Epoch 9, Loss: 0.775
Epoch 10, Loss: 0.754
Epoch 11, Loss: 0.720
Epoch 12, Loss: 0.699
Epoch 13, Loss: 0.675
Epoch 14, Loss: 0.656
Epoch 15, Loss: 0.640
Epoch 16, Loss: 0.624
Epoch 17, Loss: 0.605
Epoch 18, Loss: 0.592
Epoch 19, Loss: 0.582
Epoch 20, Loss: 0.567
Training finished.


In [75]:
total_params = 0
for name, param in net.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.numel()}")
        total_params += param.numel()
print(f"Total number of parameters: {total_params}")


conv1.weight: 864
conv1.bias: 32
bn1.weight: 32
bn1.bias: 32
conv2.weight: 18432
conv2.bias: 64
bn2.weight: 64
bn2.bias: 64
conv3.weight: 73728
conv3.bias: 128
bn3.weight: 128
bn3.bias: 128
fc1.weight: 1048576
fc1.bias: 512
fc2.weight: 5120
fc2.bias: 10
Total number of parameters: 1147914


In [76]:
def evaluate_model(model, dataloader):
    model.eval()
    predictions = []
    ground_truth = []
    
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            predictions.extend(predicted.cpu().numpy())
            ground_truth.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(ground_truth, predictions)
    precision = precision_score(ground_truth, predictions, average='weighted')
    recall = recall_score(ground_truth, predictions, average='weighted')
    f1 = f1_score(ground_truth, predictions, average='weighted')
    
    return accuracy, precision, recall, f1

test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(net, testloader)

print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Precision: {test_precision:.4f}')
print(f'Test Recall: {test_recall:.4f}')
print(f'Test F1 Score: {test_f1:.4f}')


Test Accuracy: 0.8137
Test Precision: 0.8160
Test Recall: 0.8137
Test F1 Score: 0.8121
